# loading data and insights 

In [2]:
import pandas as pd
import numpy as np

# Load dataset 
df = pd.read_csv("/kaggle/input/drug-labels-and-side-effects-dataset-1400-records/realistic_drug_labels_side_effects.csv")

In [4]:
# Basic info
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nMissing values per column:\n", df.isnull().sum())


Shape: (1436, 15)

Columns: ['drug_name', 'manufacturer', 'approval_year', 'drug_class', 'indications', 'side_effects', 'dosage_mg', 'administration_route', 'contraindications', 'warnings', 'price_usd', 'batch_number', 'expiry_date', 'side_effect_severity', 'approval_status']

Missing values per column:
 drug_name               0
manufacturer            0
approval_year           0
drug_class              0
indications             0
side_effects            0
dosage_mg               0
administration_route    0
contraindications       0
warnings                0
price_usd               0
batch_number            0
expiry_date             0
side_effect_severity    0
approval_status         0
dtype: int64
